In [1]:
import os
import numpy as np
import mujoco
import cv2
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
from base64 import b64encode
from IPython.display import HTML, display
from reflex.reflex_interface import myoLeg_reflex
from utils.config_parser import load_params_and_create_testenv, print_config_summary
from processing.exo_visualization import ExoVisualizer

def show_video(video_path, video_width=1000):
    """Display video in the notebook."""
    video_file = open(video_path, "rb").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f'''
    <video autoplay width="{video_width}" controls style="max-width: 100%;" controlsList="fullscreen">
        <source src="{video_url}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    ''')

def standardize_terrain_dimensions(env, slope_deg):
    """Ensure the heightfield dimensions match the visual ground plane."""
    if slope_deg != 0:
        try:
            ground_id = env.env.sim.model.geom_name2id('ground-plane')
            # Set the heightfield to match the plane dimensions
            env.env.sim.model.hfield_size[0, 0] = env.env.sim.model.geom_size[ground_id][0]
            env.env.sim.model.hfield_size[0, 1] = env.env.sim.model.geom_size[ground_id][1]
            env.env.forward()
            print("Successfully standardized terrain dimensions.")
        except Exception as e:
            print(f"Warning: Could not standardize terrain dimensions. Error: {e}")

MyoSuite:> Registering Myo Envs


In [2]:
# --- Load from Optimization Results ---
LOAD_FROM_FILE = True
notebook_dir = os.getcwd()
PARAMS_FILE_PATH = os.path.join(notebook_dir,"results", "exo_4param_tutorial", "myorfl_Kine_2D_1_25_2025Jul25_1827_None_BestLast.txt")
print(f"Loaded Parameters: {os.path.exists(PARAMS_FILE_PATH)}")

# --- Manual Configuration Settings ---
SIMULATION_TIME = 5      # seconds
SLOPE_DEG = 0             # env slope degrees
MODEL = "tutorial"           # Options: dephy, hmedi, humotech, osl, baseline
EXO_BOOL = False           # Enable or disable exoskeleton
USE_4PARAM_SPLINE = False # Use 4-parameter spline for exoskeleton
N_POINTS = 4              # Number of points for n-point spline
MAX_TORQUE = 100          # Maximum exoskeleton torque
CONSTANT_HEIGHT_CAMERA = False # Keep camera at a fixed height

# --- Output Directory Setup ---
output_folder = "sample_run_output"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_output_folder = os.path.join(output_folder, timestamp)
os.makedirs(run_output_folder)
print(f"Outputs will be saved to: {run_output_folder}")

Loaded Parameters: True
Outputs will be saved to: sample_run_output\20250729_143853


In [3]:
if LOAD_FROM_FILE:
    if not os.path.exists(PARAMS_FILE_PATH):
        raise FileNotFoundError(f"Parameter file not found: {PARAMS_FILE_PATH}")
    
    results_dir = os.path.dirname(PARAMS_FILE_PATH)
    filename = os.path.basename(PARAMS_FILE_PATH)
    bat_files = [f for f in os.listdir(results_dir) if f.endswith('.bat')]
    
    if not bat_files:
        raise FileNotFoundError(f"No .bat configuration file found in directory: {results_dir}")
    
    bat_file_path = os.path.join(results_dir, bat_files[0])
    
    env, config, _ = load_params_and_create_testenv(
        results_dir=results_dir,
        filename=filename,
        bat_file_path=bat_file_path,
        sim_time=SIMULATION_TIME
    )
    print_config_summary(config, title="Loaded Configuration")
    
else:
    # --- Uses the Manual Settings from previous cell ---

    config = {
        'mode': '2D', 'init_pose': 'walk_left', 'delayed': False,
        'slope_deg': SLOPE_DEG,
        'model': MODEL,
        'exo_bool': EXO_BOOL,
        'use_4param_spline': USE_4PARAM_SPLINE,
        'n_points': N_POINTS,
        'max_torque': MAX_TORQUE
    }
    
    if config['exo_bool']:
        spline_params = 4 if config['use_4param_spline'] else (config['n_points'] * 2)
    else:
        spline_params = 0
    control_params = np.ones(77 + spline_params)
    
    env = myoLeg_reflex(
        sim_time=SIMULATION_TIME,
        control_params=control_params,
        **config
    )
    print_config_summary(config, title="Manual Configuration")

env.reset()

if env.slope_deg != 0:
    standardize_terrain_dimensions(env, env.slope_deg)

print("\nEnvironment initialized.")

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    

Loaded Configuration:
  sim_time            : 5
  mode                : 2D
  init_pose           : walk_left
  slope_deg           : 0.0
  delayed             : False
  exo_bool            : True
  fixed_exo           : False
  n_points            : 0
  use_4param_spline   : True
  max_torque          : 100.0
  model               : tutorial
  model_path          : None


Environment initialized.


In [4]:
exo_visualizer = ExoVisualizer()

frames_regular = []
frames_exo = []
timesteps = int(SIMULATION_TIME / env.dt)

env.reset()

# Initialize free camera
free_cam = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(env.env.sim.model.ptr, free_cam)

# --- Tracking Camera Setup ---
camera_speed = 1.25
slope_angle_rad = np.radians(env.slope_deg)
start_position = env.env.sim.data.body("pelvis").xpos.copy()

distance_traveled = 0.0
camera_pos = start_position.copy()
initial_height_offset = 0

# Adjust starting camera height for slopes
if env.slope_deg < 0:
    initial_height_offset = abs(np.tan(slope_angle_rad)) * 20
    camera_pos[2] = 0.8 + initial_height_offset
else:
    camera_pos[2] = 0.8

free_cam.distance = 3.5
free_cam.azimuth = 90
free_cam.elevation = -10 
free_cam.lookat = camera_pos.copy()

# --- Exoskeleton Control Tracking Variables ---
exo_data = {
    'torque_r': [], 'torque_l': [],
    'stance_percent_r': [], 'stance_percent_l': [],
    'state_r': [], 'state_l': []
}

for i in range(timesteps):
    # --- Update Camera Position ---
    distance_traveled += camera_speed * env.dt
    camera_pos[0] = start_position[0] + distance_traveled
    
    # Calculate height change based on slope and distance
    height_change = (camera_pos[0] - start_position[0]) * np.tan(slope_angle_rad) * 0.2
    
    if not CONSTANT_HEIGHT_CAMERA:
        if env.slope_deg < 0:
            plateau_transition = min(1.0, i / 30)
            camera_pos[2] = 0.8 + initial_height_offset + (height_change * plateau_transition)
        else:
            camera_pos[2] = 0.8 + height_change
            
    # Update lookat target to follow model's lateral movement
    pelvis_pos = env.env.sim.data.body("pelvis").xpos.copy()
    lookat_pos = camera_pos.copy()
    lookat_pos[1] = pelvis_pos[1] 
    free_cam.lookat = lookat_pos

    # --- Capture Exoskeleton Control Data ---
    exo_control_data = exo_visualizer.get_exo_control_data(env)
    
    # Store data for this timestep
    exo_data['torque_r'].append(exo_control_data['torque_r'])
    exo_data['torque_l'].append(exo_control_data['torque_l'])
    exo_data['stance_percent_r'].append(exo_control_data['stance_percent_r'])
    exo_data['stance_percent_l'].append(exo_control_data['stance_percent_l'])
    exo_data['state_r'].append(exo_control_data['state_r'])
    exo_data['state_l'].append(exo_control_data['state_l'])

    # --- Render Frame ---
    frame = env.env.sim.renderer.render_offscreen(camera_id=free_cam, width=1920, height=1072)
    frames_regular.append(frame)
    
    # --- Create Exoskeleton Visualization Frame (0.5x speed) ---
    if env.exo_bool:
        overlay = exo_visualizer.create_overlay(
            env, 
            exo_control_data['torque_r'], 
            exo_control_data['torque_l'],
            exo_control_data['stance_percent_r'], 
            exo_control_data['stance_percent_l'],
            exo_control_data['state_r'], 
            exo_control_data['state_l']
        )
        frame_with_overlay = exo_visualizer.overlay_on_frame(frame, overlay)
        frames_exo.append(frame_with_overlay)
    else:
        frames_exo.append(frame)

    # --- Step Simulation ---
    _, _, is_done = env.run_reflex_step_Cost()
    if is_done:
        print(f"Simulation stopped at timestep {i}")
        break

video_path_regular = os.path.join(run_output_folder, "simulation_regular.mp4")
imageio.mimwrite(video_path_regular, frames_regular, fps=100, output_params=['-pix_fmt', 'yuv420p'])

video_path_exo = os.path.join(run_output_folder, "simulation_with_exo_control.mp4")
imageio.mimwrite(video_path_exo, frames_exo, fps=50, output_params=['-pix_fmt', 'yuv420p'])  # 0.5x speed

print(f"Regular video saved to: {video_path_regular}")
print(f"Exoskeleton visualization video (0.5x speed) saved to: {video_path_exo}")

display(show_video(video_path_regular))

Regular video saved to: sample_run_output\20250729_143853\simulation_regular.mp4
Exoskeleton visualization video (0.5x speed) saved to: sample_run_output\20250729_143853\simulation_with_exo_control.mp4
